# Practical tips for finetuning LLMs using LoRA

https://magazine.sebastianraschka.com/p/practical-tips-for-finetuning-llms

# Summary

The main takeaways discussed in this article are the following:

1. **LoRA consistency.** Despite the inherent randomness of LLM training (or when training models on GPUs in general), the outcomes remain remarkably consistent across multiple runs.

2. QLoRA presents a trade-off that might be worthwhile if you are constrained by GPU memory. It offers 33% memory savings at the cost of a 39% increase in runtime.

3. When finetuning LLMs, the choice of optimizer shouldn't be a major concern. While SGD on its own is suboptimal, there's minimal variation in outcomes whether you emply AdamW, SGD with a scheduler, or AdamW with a scheduler.

4. While Adam is often labeled a memory-intensive optimizer due to its introduction of two new parameters for every model parameter, this doesn't significantly affect the memory demands of the LLM. This is because the majority of the memory is allocated for large matrix multiplications rather than retaining extra parameters.

5. For static datasets, iterating multiple times, as done in multi-epoch training, might not be benefitial. It often deteriorates the results, probaly due to overfitting.

6. If you are incorporating LoRA, ensure it is applied across all layers, not just to the Key and Value matrices, to maximize model performance.

7. Adjusting the LoRA rank is essential, and so is selecting an apt `alpha` value. A good heuristic is setting `alpha` at twice the rank's value.

8. 7 billion parameter models can be finetuned efficiently within a few hours on a single GPU possessing 14 GB of RAM. With a static dataset, optimizing an LLM to excel across all benchmark tasks is unattainable. Addressing this requires diverse data sources, or perhaps LoRA might not be the ideal tool.

In addition, this article will answer ten common questions around LoRA:

* Q1: How Important is the Dataset?

* Q2: Does LoRA Work for Domain Adaptation?

* Q3: How Do You Select the Best Rank?

* Q4: Does LoRA Need to Be Enabled for All Layers?

* Q5: How To Avoid Overfitting?

* Q6: What about Other Optimizers?

* Q7: What Other Factors Influence Memory Usage?

* Q8: How Does it Compare to Full Finetuning and RLHF?

* Q9: Can LoRA Weights be Combined?

* Q10: What about Layer-wise Optimal Rank Adaptation?

# 1 - LoRA consistency

Running multiple experiments with LoRA, I found that the benchmark results are surprinsgly consistent across the different runs despite the inherent randomness of LLM training or when training models on GPUs in general. This is a good basis for additional comparison studies.

<table>
    <tr>
        <td><img src="./images_4/lora_consistency.webp" width="700"/></td>
    </tr>
</table>

The results were obtained with default setting using a small $r=8$. The experimental details can be found [here](https://lightning.ai/pages/community/lora-insights/).

# 2 - QLoRA compute-memory trade-offs

QLoRA by [Dettmers et al. (2023)](https://arxiv.org/abs/2305.14314) short for quantized LoRA, is technique that further reduces memory usage during finetuning. During backpropagation, QLoRA quatizes the pretrained weights to 4-bit precision and uses paged optimizers to handle memory spikes-

Indeed, I found that one can save 33% of GPU memory when using QLoRA. However, this comes at a 39% increased training runtime caused by the additional quantization and dequantization of the pretrained model weights in QLoRA.

Default LoRA with 16-bit brain floating point precision:

* Training time: 1.85 h
* Memory used: 21.33 GB

QLoRA with 4-bit *Normal Floats*:

* Training time: 2.79 h
* Memory used: 14.18 GB

Moreover, I found that the modeling performance was barely affected, which makes QLoRA a feasible alternative to regular LoRA training to work around the common GPU memory bottleneck.

<table>
    <tr>
        <td><img src="./images_4/lora_vs_qlora.webp" width="700"/></td>
    </tr>
</table>

# 3 - Learning Rate Schedulers

Learning rate schedulers lower the learning rate throughout the trainig to optimize convergence and avoid overshooting the loss minima.

Cosine annealing is a learning rate scheduler that adjusts the learning rate following a cosine curve. It starts with a high learning rate, which then decreases smoothly, approaching zero in a cosing-like manner. A commonly used variant is the half-cycle variant, where only a half-cosine cycle is completed over the course of training, as shown in the figure below.

<table>
    <tr>
        <td><img src="./images_4/cosine_annealing.webp" width="300"/></td>
    </tr>
</table>

As part of my experiments, I added a cosine annealing scheduler to the LoRA finetuning scripts and observed that it improved the SGD performance noticeably. However, it has less impact on Adam and AdamW optimizers and makes almost no difference

<table>
    <tr>
        <td><img src="./images_4/lora_lr_scheduler.webp" width="700"/></td>
    </tr>
</table>

The potential advantages of SGD over Adam are discussed in the next section.

# 4 - Adam vs SGD

Adam and AdamW optimizers remain popular choices in deep learning even though they are memory-intensive when we are working with large models. The reaon is that Adam optimizers maintain two moving averages for each model parameter: the first moment (mean) of the gradients and the second moment (uncentered variance) of the gradients. In other words, Adam optimizers store two additional values for each single model aprameter in memory. If we are working with a 7B parameter model, that's an extra 14B parameters to track during training.

SGD optimizers don't need to track any additional parameters during training, so a question is: what advantage does swapping Adam by SGD have on the peak memory requirements when training LLMs?

In my experiments, training a 7B parameter LLama 2 model trained with AdamW and LoRA defaults (`r=8`) required 14.18GB of GPU memory. Training the same model with SGD instead required 14.15 GB of GPU memory. In other words, the savings (0.03 GB) were minimal.

**Why are the memory savings so small?** That is because with LoRA, we only have a small number of trainable parameters. For instance, if `r=8`, we have 4,194,304 trainable LoRA parameters out of all 6,738,415,616 parameters in a 7B Llama 2 model (~0.06% of the total).

If we just look at the bare numbers, 4 million trainable parameters still sound like  alot, but if we do the math, we only have 4,194,304 × 2 × 16 bit = 134.22 megabits = 16.78 megabytes. (We observed a 0.03 Gb = 30 Mb difference since there is an additional overhead in storing and copying optimizer states.) The 2 represents the number of extra parameters that Adam stores, and the 16-bit refers to the default precision for the model weights. 

However, if we increase the LoRA `r` to 256, something I've done in later experiments, the difference between Adam and SGD optimizers becomes more noticeable:

* 17.86 GB with AdamW
* 14.46 GB with SGD

As a takeaway, swapping Adam optimizers with SGD may not be worthwhile when LoRA's r is small. However, it may be worthwhile when we are increasing r. 

# 5 - Multiple Training Epochs

In conventional deep learning, we often iterate over a training set multiple times -- an iteration over the training set is called an epoch. It's common to run hundreds of training epochs when trainig convolutional neural networks, for example. Is multi-epoch training useful for instruction finetuning as well?

When I increased the number of iterations for the [50k-example Alpaca instruction finetuning dataset](https://github.com/tatsu-lab/stanford_alpaca) by a factor of two (analogous to 2 training epochs), I noticed a decline in model performance.

<table>
    <tr>
        <td><img src="./images_4/multiple_training_epochs.webp" width="700"/></td>
    </tr>
</table>

The takeaway is that multi-epoch trainig might not benefit instruction finetuning since it can deteriorate the results. I observed the same with the 1k-example LIMA dataset. This performance decline is likely due to increased overfitting, which warrants additional investigation.

# 6 - Enable LoRA for more layers

The tables above showed experiments where LoRA was only enabled for select weight matrices, i.e., the Key and Value weight matrices in each transformer layer. In addition, we can also enable LoRA for the Query weight matrices, the projection layers, the other linear layers between the multihead attention blocks, and the linear output layer.

<table>
    <tr>
        <td><img src="./images_4/lora_parameters.webp" width="300"/></td>
    </tr>
</table>

If we enable LoRA for all these additional layers, we increase the number of trainable parameters by a factor of 5, from 4,194,304 to 20,277,248

<table>
    <tr>
        <td><img src="./images_4/lora_more_layers.webp" width="700"/></td>
    </tr>
</table>

However, a limitation of my experiment is that I only explored two settings:

1. LoRA for only the query and value weight matrices enabled
2. LoRA for all layers enabled. 

It might be worthwhile exploring the other combinations in future experiments. For example, it would be interesting to know whether activating LoRA for the projection layer is actually beneficial.

# 7 - Balancing LoRA hyperparameters: `r` and `alpha`

As the original LoRA paper outlines, LoRA introduces an additional scaling coefficient for applying the LoRA weights to the pretrained weights during the forward pass. The scaling involves the rank parameter `r`, which we discussed earlier, as well as another hyperparameter`alpha` that is applied as follows:

```python

scaling = alpha / r
weight += (lora_B @ lora_A) * scaling

```

As we can see in the code formula above, the larger the `alpha`, the larger the influence of the LoRA weights.

Previous experiments used `r=8` and `alpha=16`, which resulted in a 2-fold scaling. **Choosing `alpha` as two times `r` is a common rule of thumb when using LoRA for LLMs**, but I was curious if this still hold for larger `r` values. In other words, `alpha = 2 * rank` really seems to be a sweet spot. 

However, in this specific combination of model and dataset, I observed that `r=256` and `alpha=128` (i.e., 0.5-fold scaling) showed a better perfomance:

<table>
    <tr>
        <td><img src="./images_4/lora_alpha.webp" width="700"/></td>
    </tr>
</table>

(I experimented with `r=32`, `r=64`, `r=128`, and `r=512` but omitted the results for clarity as `r=256` resulted in the best performance)

Choosing `alpha` as two times as large as `r` may often result in the best outcomes, but it may also not hurt to experiment with different ratios.

# Answers to common questions

## Q1: How important is the dataset?

The dataset can be critical. I used the Alpaca dataset, which consists of 50k training examples, for my experiments. I chose this dataset because it's quite popular, and experimenting with different datasets was out of scope due to the already extensive length of the article.

However, it's worth noting that Alpaca is a synthetic dataset that was generated by querying an old version of ChatGPT and is probably not the best by today's standards.

Dataquality can be very important. For example, according to [LIMA: Less is more important for alignment paper](https://arxiv.org/abs/2305.11206), a curated dataset consisting of only 1k examples makes Llama outperform a version of Llama finetuned on Alpaca.

<table>
    <tr>
        <td><img src="./images_4/lima_vs_alpaca.webp" width="700"/></td>
    </tr>
</table>

Using the best configuration (`r=256`, `alpha=512`) on LIMA, I got similar, if not better, performance than the 50x larger Alpaca dataset.

<table>
    <tr>
        <td><img src="./images_4/lora_lima_alpaca.jpg" width="700"/></td>
    </tr>
</table>

## Q2: Does LoRA work for domain adaptation?

Unfortunately, I don't have a good answer to this question. As a rule of thumb, knowledge is usually absorbed from the pretraining dataset. Instruction finetuning is generally more about helping or guiding the LLM towards following instructions.

However, it's worth noting that if memory is a convern, LoRA can also be used for further pretrainig existing pretrained LLMs on domain-specific datasets.


## Q3: How do you select the best rank?

Unfortunately, I don't have any good heuristic for selecting a good `r` and think that **it's a hyperparameter that needs to be explored for each LLM and each dataset**.

I suspect that choosing an `r` that is too large could result in more overfitting. On the other hand, a small `r` may not be able to capture diverse tasks in a dataset. In other words, I suspect that the more diverse the tasks in the dataset, the larger the `r` should be. For example, if I only want a model that carries out basic 2-digit arithmetic, then a tiny `r` might already be sufficient. However, this is only a hypothesis and would require additional investigation.

## Q4: Does LoRA need to be enabled for all layers?

I only explored two settings: (1) LoRA for only the query and value weight matrices enabled, and (2) LoRA for all layers enabled. It might be worthwhile exploring the other combinations in future experiments. For example, it would be interesting to know whether activating LoRA for the projection layer is actually beneficial.

<table>
    <tr>
        <td><img src="./images_4/lora_parameters.webp" width="300"/></td>
    </tr>
</table>

For instance, if we consider the various settings (`lora_query`, `lora_key`, `lora_value`, `lora_projection`, `lora_mlp`, and `lora_head`), that is `2^6 = 64` combinations to explore. This exploration would be an interesting topic for future studies.

## Q5: How to avoid overfitting?

Generally, a larger `r` can lead to more overfitting because it determines the number of trainable parameters. If a model suffers from overfitting, decreasing `r` or increasing the dataset size are the first candidates to explore. Moreover, you could try to increase the weight delay rate in AdamW or SGD optimizers, and you can consider increasing the dropout value for LoRA layers.

The LoRA dropout parameter that I haven't explored in my experiments (I used a fixed dropout rate of 0.05), is an interesting topic for future investigations.

## Q6: What about other optimizers?

Other interesting optimizers for LLMs are worth exploring in the future. One such optimizer is [Sophia: A scalable stochastic second-order optimizer for language model pre-trainig](https://arxiv.org/abs/2305.14342), which was published in 05-2023.

Compared to Adam, Sophia is 2× faster, and models trained with Sophia can achieve better modeling performance, according to the paper. In a nutshell, Sophia normalizes the gradients by gradient curvature instead of gradient variance, as in Adam.

## Q7: What other factors influence memory usage?

Besides precision and quantization settings, the model size, the batch size, and the number of trainable LoRA parameters, the dataset can also influence memory usage.

Note that LLama 2 has a context size of 4048. It can process sequences of up to 4048 tokens at once. However, shorter training sequences can result in substantial memory savings due to the masking of future tokens.

For example, the Alpaca dataset is relatively small, with a maximum length of 1304 tokens.

<table>
    <tr>
        <td><img src="./images_4/alpaca_seq_length_distribution.webp" width="400"/></td>
    </tr>
</table>

When I experimented with other datasets that had lengths of up to 2048 tokens, I noticed that the memory usage went up from 17.86GB to 26.96GB.

## Q8: How does it compare to full finetuning and RLHF?

Full finetuning required at least 2 GPUs and was completed in 3.5 h using 36.66 GB on each GPU. However, the benchmark results were not very good, likely due to overfitting or suboptimal hyperparameters.

<table>
    <tr>
        <td><img src="./images_4/lora_vs_full_finetuning.webp" width="700"/></td>
    </tr>
</table>

## Q9: Can LoRA weights be combined?

Yes, it's possible to combine multiple sets of LoRA weights. During training, we keep the LoRA weights separate from the pretrained weights and add them during each forward pass.

However, if you have a real-world application with many set so of LoRA weights, for example, one set for each application customer, it makes sense to store these weights separately to save disk space. However, it's possible to merge the pretrained weights with the LoRA weights after training to create a single model. This way, we don't have to apply the LoRA weights in each forward pass:

```python
weight += (lora_B @ lora_A) * scaling
```

Instead, we apply the weight update as shown above and save the merged (added) weights.

similarly, we can keep adding multiple LoRA weight sets:

```python
weight += (lora_B_set1 @ lora_A_set1) * scaling_set1
weight += (lora_B_set2 @ lora_A_set2) * scaling_set2
weight += (lora_B_set3 @ lora_A_set3) * scaling_set3
...
```

I have yet to do experiments to evaluate the performance of such an approach, but this is technically already possible via the [scripts/merge_lora.py script provided in Lit-GPT](https://github.com/Lightning-AI/lit-gpt/blob/main/scripts/merge_lora.py).

## Q10: What about layer-wise optimal rank adaptation?

For simplicity, we usually train deep neural networks with the same learning rate for each layer, and the learning rate is a hyperparameter that we need to optimize. To take it further, we can also choose a different learning rate for each layer ([in PyTorch, this is not too complicated](https://kozodoi.me/blog/20220329/discriminative-lr#:~:text=The%20implementation%20of%20layer%2Dwise,with%20the%20corresponding%20learning%20rates.)). However, **it's rarely done in practice because it adds additional overhead**, and there are usually already so many knobs to tune when trianing deep neural networks.

Analogous to choosing different learning rates for different layers, we can also choose different LoRA ranks for different layers. I haven't found any experiments on this, but a document that details this approach is [Layer-wise optimal rank adaptation](https://medium.com/@tom_21755/llm-optimization-layer-wise-optimal-rank-adaptation-lora-1444dfbc8e6a). In theory, this sounds like a good idea. However, in practice, it also adds an extensive number of choices when optimizing hyperparameters.